## Gaussian Mixuture

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df=pd.read_csv("./Gaussian_Mixture.csv")

## Trabajando con el K-Means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df.head()

In [ ]:

kmeans=KMeans(n_clusters=??)

In [ ]:
kmeans.fit(df)

In [ ]:
pred=kmeans.predict(df)

In [ ]:
df["Cluster"]=pred

In [ ]:
color=
for k in range(¿¿,??):
    df_aux=df[df["Cluster"]==k]
    plt.scatter(df_aux["Weight"],df_aux["Height"],c=color[k])
plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm=GaussianMixture(n_components=??)
gm.fit(df.drop(columns="Cluster"))

In [ ]:
## Media de cada 
gm.means_

In [ ]:
## Media de covarianza 
gm.covariances_

In [ ]:
## Pesos
gm.weights_

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
## Oh no podemos pq no tenemos target
silhouette_score()

In [ ]:
df["Cluster"]=gm.predict(df.drop(columns="Cluster"))

In [ ]:
color=
for k in range(0,4):
    df_aux=df[df["Cluster"]==k]
    plt.scatter(df_aux["Weight"],df_aux["Height"],c=color[k])
plt.show()